In [ ]:
#load in Numpy binaries from file

In [1]:
import numpy as np

In [2]:
with open('movie-xids.npy', 'rb') as f:
    Xids = np.load(f, allow_pickle=True)
with open('movie-xmask.npy', 'rb') as f:
    Xmask = np.load(f, allow_pickle=True)
with open('movie-labels.npy', 'rb') as f:
    labels = np.load(f, allow_pickle=True)

In [3]:
Xids.shape

(156060, 512)

In [4]:
labels

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [ ]:
#create a TF dataset object

In [5]:
import tensorflow as tf

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

In [7]:
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.int32, name=None), TensorSpec(shape=(512,), dtype=tf.int32, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [ ]:
#rearrange the dataset format to a two-item tuple ({input_ids, attention_mask}, outputs)

In [9]:
def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

In [10]:
#use the dataset map method to apply the transformation
dataset = dataset.map(map_func)

In [11]:
#show the dataset sample format
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int32, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [ ]:
#shuffle the data and batch it

In [12]:
batch_size = 16

In [13]:
#take batch sizes of 16 and drop any samples that don't fit evenly into chunks of 16
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

In [14]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [ ]:
#split the data into training and validation sets

In [15]:
#use the take and skip method, creating an 90-10 split
split = 0.9

In [17]:
size = int(Xids.shape[0] / batch_size * split)

In [18]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [ ]:
#free up memory
del dataset

In [ ]:
#save the datasets

In [20]:
tf.data.experimental.save(train_ds, 'train')
tf.data.experimental.save(val_ds, 'val')

Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


In [21]:
train_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [22]:
val_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [23]:
train_ds.element_spec == val_ds.element_spec

True